In [83]:
import json
import numpy as np

In [96]:
sources = ["result_llama-2-7b-chat_gsm8k_decomp_llama-2-7b-chat_direct.json",
          'result_llama-2-7b-chat_gsm8k_decomp_llama-2-7b-chat_mathreg.json']

In [97]:
eval_mode = True
incorrect = []
for src in sources:
    cur_incorrect = []
    print(src)
    if 'shot' in src:
        # eval mode for the baseline methods
        eval_mode = False
    num_correct = 0
    num_tot = 0
    with open(src, "r") as f:
        res = json.load(f)
        for i, item in enumerate(res):
            if eval_mode:
                gt = item["final_ans"]
                out = item["answer"]
            elif eval_mode == False:
                # gsm8k
                if 'result_test_with_ids_llama' in src:
                    gt = item['answer']
                    out = item['model_answer']
                # multiarith
                elif 'result_test_llama' in src:
                    gt = item['final_ans']
                    out = item['model_answer']
    
            if "gsm8k" in src or 'result_test_with_ids_llama' in src:
                gt = gt.split("#### ")[-1]
            try:
                if abs(float(gt) - float(out)) <= 1e-5:
                    num_correct += 1
                else:
                    cur_incorrect.append(i)
            except:
                cur_incorrect.append(i)
                num_correct += 0
            
            num_tot += 1
    incorrect.append(cur_incorrect)
    print(num_correct / num_tot)
    print(" ")

result_llama-2-7b-chat_gsm8k_decomp_llama-2-7b-chat_direct.json
0.18802122820318423
 
result_llama-2-7b-chat_gsm8k_decomp_llama-2-7b-chat_mathreg.json
0.18802122820318423
 


In [98]:
misclassify = []
for wrong in incorrect[0]:
    if wrong not in incorrect[1]:
        misclassify.append(wrong)
misclassify = np.array(misclassify)

In [99]:
len(misclassify)

83

In [92]:
file_path = 'result_llama-2-7b-chat_gsm8k_decomp_llama-2-13b-chat_direct.json'

with open(file_path, 'r') as file:
    data = json.load(file)
data = np.array(data)
for question in data[misclassify[2:3]]:
    print(question['sub_questions_answers'][question['sub_questions_answers'].index("Question 5:"):])

Question 5: Kyle bought last year's best-selling book for $19.50. This is with a 25% discount from the original price. What was the original price of the book?
Question 5.1: What was the original price of the book before the discount?
Answer 5.1: The original price of the book before the discount was $19.50 / 0.75 = $26.00.

Question 5.2: Now we can answer the question: What was the original price of the book?
Answer 5.2: The original price of the book was $26.00. The answer is $26.00.



In [93]:
file_path = 'result_llama-2-7b-chat_gsm8k_decomp_llama-2-13b-chat_mathreg.json'

with open(file_path, 'r') as file:
    data = json.load(file)
data = np.array(data)
for question in data[misclassify[2:3]]:
    print(question['sub_questions_answers'][question['sub_questions_answers'].index("Question 5:"):])

Question 5: Kyle bought last year's best-selling book for $19.50. This is with a 25% discount from the original price. What was the original price of the book?
Question 5.1: What was the original price of the book before the discount?
Answer 5.1: The original price of the book was 25% higher than the price Kyle paid. So the original price was $19.50 * 1.25 = $24.375000 .
Question 5.2: Now we can answer the question: What was the original price of the book?
Answer 5.2: 24 dollars and 37 cents. The answer is $24.375000 .
